In [75]:
import pandas as pd
from ortools.linear_solver import pywraplp

In [76]:
solver = pywraplp.Solver.CreateSolver('GLOP')

In [77]:
df = pd.read_csv('RA.csv')
df.head()

,Região administrativa,Num de veiculos,% de veiculos,lat,lon
0,Plano Piloto,74038,86.9,-15.797422,-47.878090
1,Gama,25485,65.0,-16.018858,-48.067951
2,Taguatinga,45951,72.0,-15.803228,-48.017552
3,Brazlândia,8798,56.1,-15.622604,-48.134750
4,Sobradinho,13364,72.9,-15.633557,-47.829898


## Variáveis do problema

In [78]:
N = len(df)                                 # Número de RAs
V = df['Num de veiculos'].to_numpy()        # Número de veículos
D = pd.read_csv('distances.csv', header=None).values     # Distância entre as RAs

cities = df['Região administrativa'].to_numpy()

In [79]:
cap_min = 10000         # Capacidade min de atendimento de cada drive thru
cap_max = 20000         # Capacidade max de atendimento de cada drive thru
doses = 200000          # Número de doses de vacina disponíveis
qtde_pessoas = 2        # Quantidade média de pessoas vacinadas por carro
dist_max = 15           # Distância máxima entre dois postos drive thru

## Variável alvo

Quantidade de drive thrus por Região Administrativa do DF

- **Limite mínimo**: 0
- **Limite máximo**: 2

In [80]:
X = []

for i in range(N):
    qt_vacinantes = V[i] * qtde_pessoas
    qt_max = int(min(cap_max, qt_vacinantes))

    if qt_vacinantes < cap_min:
        X.append(solver.IntVar(0, 0, f'X{i}'))
    else:
        X.append(solver.IntVar(0, qt_max, f'X{i}'))

## Restrições

1. O número máximo de doses distribuídas deve ser menor do que a qtde disponível


$$ \sum_{i=0}^{N} X_{i}  = doses$$

In [81]:
ct = solver.Constraint(doses, doses, 'numero max de doses')
for i in range(N):
    ct.SetCoefficient(X[i], 1)

2. Uma região só receberá um drive thru se a quantidade de vacinantes (Qtde de carros [V] * média de vacinantes por carro [q]) for maior do que a capacidade de vacinação do posto

$$\sum X_{i} \frac{c}{qV_{i}} \leq 1$$

In [82]:
# for i in range(N):
#     ct = solver.Constraint(cap_min, cap_max, f'Cidade {cities[i]} - dist min')
#     for j in range(N):
#         var = 0
#         if D[i][j] <= dist_max:
#             var = 1
#         ct.SetCoefficient(X[i], var)

## Função objetivo

Minimizar o custo de distribuição das vacinas

Aqui, é considerado o custo de transporte das vacinas até o posto (considerando que o centro de distribuição está localizado no Plano Piloto) e o custo de armazenamento das vacinas (considerando o preço do aluguel médio de cada região administrativa)

$$min \left ( \sum_{i=0}^{N} X_{i} (A_{i} + gD_{i}) \right )$$

In [83]:
objective = solver.Objective()

for i in range(N):
    objective.SetCoefficient(X[i], 1)
objective.SetMaximization()

## Resultado

In [84]:
status = solver.Solve()

In [85]:
if status == pywraplp.Solver.OPTIMAL:
    print('Valor objetivo =', solver.Objective().Value())
    for i in range(N):
        n = X[i].solution_value()
        if n:
            print(f"{cities[i]}: {n} vacinas")
else:
    print('O problema não possui solução ótima')

O problema não possui solução ótima
